## Downloading Packages
This step is required to be carried out in the beginning of new session.

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# coding: utf-8
from __future__ import print_function
import json
from ibm_watson import DiscoveryV1
import json
!pip install ibm_watson
!pip install stackapi
!pip install twython

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
  Created wheel for stackapi: filename=StackAPI-0.1.12-cp36-none-any.whl size=5668 sha256=8938da54da8532e9d8eef7e9c2122c6ac7d78b542f2f014eee615956f8b1ef4b
  Stored in directory: /root/.cache/pip/wheels/16/bf/a5/56362daf788c5ce88244796b26d3e7d6fec649f5e77aff694c
Successfully built stackapi
  Created wheel for twython: filename=twython-3.7.0-cp36-none-any.whl size=31997 sha256=115865f4b3e6c898fa2d5af0d8fe9a994d03736fd7487bed0450e7cc05d5434a
  St

###Importing Modules

In [0]:
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from stackapi import StackAPI

##Defining Constants

In [0]:
k = 10
SITE = StackAPI('stackoverflow')

#Problem 1
##To identify most relevant questions to a query (text similarity)

###Get query from the user
This is the same query that user would enter in the search field of StackOverflow.


In [4]:
query = input("Enter the query: ")

Enter the query: Node - how to run node app.js?


###Extracting tags from the query
Extract keywords from the query into a list and then shorten the list.

NOTE: This can be further improved by extracting tags using StackOverflow tags dataset.

In [5]:
#Remove punctuations
text = re.sub('[^a-zA-Z]', ' ', query)
#Convert to lowercase
text = text.lower()
#Remove tags
text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
#Remove special characters and digits
text = re.sub("(\\d|\\W)+"," ",text)
#Convert to list from string
text = text.split()



#Reduce the list
stop_words = set(stopwords.words("english"))
cv = CountVectorizer(max_df=0.8, stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X = cv.fit_transform(text)
shortened_list = list(cv.vocabulary_.keys())[:10]
print("Query: " + str(query))
print("Tags: " + str(shortened_list))

Query: Node - how to run node app.js?
Tags: ['node', 'run', 'app', 'js']


###Getting questions from the tags using StackAPI

In [6]:
tags_string = " ".join(shortened_list)

#Fetching list of questions into questions_json
questions_json = SITE.fetch('search/excerpts', q=tags_string, sort='relevance', filter = '!)Q29nl5DNb5bipTWVFKCiZie')

#Printing first k questions
for question in questions_json['items']:
  print(question['title'])

Run multiple node js app on digitalocean with different ports
How to run node js app with express on my server
Proxy error 502 , run node js app on apache2 server
How can i run more than one app in node js?
Remove port from Node js website url
Cannot run node js app listening on 443
How to run AngularJS 1 app locally using Node js?
Most simple and minimalistic way to run vue js single page app with Node. (No CDN)
Cant get Firebase in Node JS app to run
Run React basic app on ubuntu using node js
how to run the express js(node js) in command line - linux
Is Node JS needed to build Angular JS app ? If yes why? Why don&#39;t we need node js in production env?
Deploy node js app on iis
repl node js problem SyntaxError: Unexpected identifier
Deploying node js and php on heroku
Expose node js app with host as localhost on kubernetes
Run jar file with node js on Microsoft Azure
Is it possible to run node js apps as a background service on android?
How can I use Gulp to run a Node JS app?
Coul

###Sorting questions based on ***text similarity*** scores
Calculate text similarity between user query and question title. 
Sort the question in order of their (cosine) scores and display them to the user. 

In [7]:
vect = TfidfVectorizer(min_df=1)

#Creating a dictionary of score and list of questions corresponding to the score
#Here, each question is represented as a dictionary of question_id, title, and body
score_question_dict = {}
for question in questions_json['items']:
  #Calculating text similairty between query and question title
  tfidf = vect.fit_transform([query, question['title']])
  #Considering cosine text similarity score
  question_score = ((tfidf * tfidf.T).A)[0,1]
  #Feeding in the dictionary
  if question_score not in score_question_dict.keys():
      score_question_dict[question_score] = [{'question_id':question['question_id'], 'title':question['title'], 'body':question['body']}]
  else: 
      score_question_dict[question_score] += [{'question_id':question['question_id'], 'title':question['title'], 'body':question['body']}]
        
#Printing question as they appear on the site with title and body
print(str(len(score_question_dict))+" Results")
for question_score in sorted(score_question_dict.keys(), reverse=True):
  for question in score_question_dict[question_score]:
    print(str(question['question_id']) + ": " + question['title'])
    print(question['body'])
    print("------------------------------------------------------")

59 Results
23612154: How to run a node.js app from name
How do you create a console app that you can run by a name instead of using node and a .js file to start the app?

Instead of...
 $ node myapp.js --out ./folder/

Can I...
 $ myapp --out ./folder/

------------------------------------------------------
42242498: How to run node.js app by grunt?
Folder structure:-

myapp

--public(directory) //here files which related to public(Angularjs,css,etc)

--server(directory) //here files which related to server

--server.js

Code



I run app by  but now i want to run this app by grunt and when i run  the it throw No &quot;connect&quot; targets found error. 

How to configure grunt file to run app?

------------------------------------------------------
30450684: How to run node webkit app in ubuntu 12.04?(Node js)
I am new nodewebkit .I have created simple application of hello world.

index.html



package.json



then i have zipped using .nw extension (sample.nw)
after all i have execute

###Get question_id from the user
Ask user to select a question from the list of questions using question_id. 

This is similar to user selecting the question (by clicking on the link) from the search results on StackOverflow website.

In [8]:
question_id = int(input('Select a question: '))

Select a question: 38868890


#Problem 2
##To identify top k solutions of the selected question (Sentiment Analysis of review content)

###Getting answers to the selected question using StackAPI

In [1]:
# service instance provides API key authentication
discovery = DiscoveryV1(
    version='2018-08-01',
    ## url is optional, and defaults to the URL below. Use the correct URL for your region.
    url='https://gateway-lon.watsonplatform.net/discovery/api',
    iam_apikey='Your API key')

#checking the list of environments
environments = discovery.list_environments().get_result()
print(json.dumps(environments, indent=2))
# print(json.dumps(news_environment_id, indent=2))



myenv_ID  = 'Your enviornment'
collection_ID = 'Your collection id'
config_ID = 'Your configuration id'
doc_ID=''
ss=0

# creating collection - has to be done only once
# discovery.create_collection(myenv_ID,'comments')

# collections dictionary
col = discovery.list_collections(myenv_ID).get_result()

NameError: ignored

In [0]:
#getting the answer json from the stack api
answers_json = SITE.fetch('questions/{ids}/answers', ids = [question_id], filter = '!*cCFgkLFk24i(9rKI)wpCc747lAHZul(ht_tD')

score_answer_dict = {}

for answer_item in answers_json['items']:
  if 'comments' in answer_item.keys():
    comments = ''
    for comment in answer_item['comments'] :
      comments = comments+' '+comment['body_markdown']
#     print(json.dumps(comments,indent = 2)) 
    
    file = open('collections.html', 'w')
    file.write(comments)
    file.close()
    
#adding the document to the collection
    meta_data = {"document_name": "name1"}
    with open('collections.html', "r") as fileinfo:
      add_doc = discovery.add_document(myenv_ID,collection_ID,file = fileinfo).get_result() 
    
    result = discovery.query(environment_id=myenv_ID,
                                 collection_id=collection_ID,
                                 query='',
                                 deduplicate=True,
                                 aggregation='filter(enriched_title.entities.type::Company).term(enriched_title.entities.text).timeslice(crawl_date,1day).term(enriched_text.sentiment.document.score)',
                                 return_fields='enriched_text.sentiment.document')
#getting the sentiment score from the IBM watson api and adding the answers according to the score to the scre_answer_dict 
    for x in result.result['results']:
      doc_ID = x['id']
      enriched_txt = x['enriched_text']   
      senti = enriched_txt['sentiment']
      doc = senti['document']
      ss=doc['score']
    if ss not in score_answer_dict.keys():
      score_answer_dict[ss] = [{'answer_id':answer_item['answer_id'], 'question_id':answer_item['question_id'], 
                                   'question_title':answer_item['title'], 'answer_body':answer_item['body_markdown']}]
    else: 
      score_answer_dict[ss] += [{'answer_id':answer_item['answer_id'], 'question_id':answer_item['question_id'], 
                                   'question_title':answer_item['title'], 'answer_body':answer_item['body_markdown']}]
    
    #deleting the created document
    discovery.delete_document(myenv_ID,collection_ID,doc_ID)
      
     

###Sorting questions based on sentiment analysis on their comments
Analyse all the comments to an answer for sentiment. Assign each answer a score based on this analysis. Sort the answers and display them to the user.  

In [20]:
   
#Display the answers to the user sorted by sentiment analysis scores
print('Question (' + str(answers_json['items'][0]['question_id']) + "): " + str(answers_json['items'][0]['title']), end='\n\n')
for answer_score in sorted(score_answer_dict.keys(), reverse=True):
  for answer in score_answer_dict[answer_score]:
    print('Answer ID: ' + str(answer['answer_id']))
    print(answer['answer_body'])
    print("------------------------------------------------------")

Question (51136727): Google App Engine Standard Node JS how to run build script?

Answer ID: 51158589
In GAE standard, installation of dependencies are automatically managed. You should add them in your `package.json`.

As [Google documentation][1] mentioned :

&gt; When you deploy your app, the Node.js runtime automatically installs all dependencies declared in your package.json file using the npm install command.

    {
      &quot;dependencies&quot;: {
        &quot;lodash&quot;: &quot;^4.0.1&quot;
      }
    }

Installation will be done during app deployment via : 

    gcloud app deploy


  [1]: https://cloud.google.com/appengine/docs/standard/nodejs/specifying-dependencies
------------------------------------------------------
